convolutional neural networks (CNN)

image classificaation

In [74]:
pip install gradio

In [75]:
from PIL import Image


In [76]:
!pip install -U gradio

In [78]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
# The ImageDataGenerator class should be imported from tensorflow.keras.preprocessing.image
from tensorflow.keras.preprocessing import image
import gradio as gr



In [79]:
url='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
cat_and_dog_file_path=os.path.join(os.getcwd(), 'cat_and_dog_filtered.zip')
cat_and_dog_file_path

'/content/cat_and_dog_filtered.zip'

In [80]:
os.makedirs('Data', exist_ok=True)
cat_and_dog_file_path=os.path.join(os.getcwd(), 'Data', 'cat_and_dog_filtered.zip')
cat_and_dog_file_path

'/content/Data/cat_and_dog_filtered.zip'

In [61]:
import urllib.request
import zipfile


In [81]:
urllib.request.urlretrieve(url, cat_and_dog_file_path)
with zipfile.ZipFile(cat_and_dog_file_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(os.getcwd(),'Data'))

In [82]:
cat_and_dog_file_path

'/content/Data/cat_and_dog_filtered.zip'

In [83]:
main_dir=os.path.join(os.getcwd(),'content')


In [65]:
train_folder = os.path.join(os.getcwd(), 'Data', 'cats_and_dogs_filtered', 'train')
test_folder= os.path.join(os.getcwd(), 'Data', 'cats_and_dogs_filtered', 'validation')

In [84]:
#train_folder=os.path.join(main_dir, 'train')
#test_folder=os.path.join(main_dir, 'validation')

now for better accuracy and classification we will use convolutional neural networks

In [85]:
CDC = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(8,(3,3),activation='relu',input_shape=(200,200,3)),
      tf.keras.layers.MaxPool2D(2,2),
      tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(200,200,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(200,200,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(500,activation='relu'),
      tf.keras.layers.Dense(1,activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [89]:
CDC.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])


In [87]:
train_gen = image.ImageDataGenerator(rescale= 1. / 255)
test_gen = image.ImageDataGenerator(rescale= 1. / 255)

In [90]:
train_folder = os.path.join(os.getcwd(), 'Data', 'cats_and_dogs_filtered', 'train')
train_dgen = train_gen.flow_from_directory(
    train_folder,
    class_mode='binary',
    target_size=(200,200),
    batch_size=32
)

Found 2000 images belonging to 2 classes.


In [91]:
test_dgen = test_gen.flow_from_directory(
    test_folder,
    class_mode='binary',
    target_size=(200,200),
    batch_size=32
)

Found 1000 images belonging to 2 classes.


In [92]:
Trained_CDC = CDC.fit(
    train_dgen,
    validation_data=test_dgen,
    epochs=5,
    steps_per_epoch=90,
    verbose=2,
    validation_steps = 50
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


90/90 - 59s - 660ms/step - accuracy: 0.5245 - loss: 0.7478 - val_accuracy: 0.5910 - val_loss: 0.6692
Epoch 2/5
90/90 - 57s - 633ms/step - accuracy: 0.5970 - loss: 0.6661 - val_accuracy: 0.5650 - val_loss: 0.6492
Epoch 3/5
90/90 - 82s - 910ms/step - accuracy: 0.6480 - loss: 0.6110 - val_accuracy: 0.5390 - val_loss: 1.1559
Epoch 4/5
90/90 - 83s - 920ms/step - accuracy: 0.6985 - loss: 0.5804 - val_accuracy: 0.7020 - val_loss: 0.5772
Epoch 5/5
90/90 - 57s - 629ms/step - accuracy: 0.7390 - loss: 0.5270 - val_accuracy: 0.6890 - val_loss: 0.5637


In [93]:
def c_d_id(i):
  i = i.resize((200,200),resample=Image.Resampling.BILINEAR)
  i_arr = image.img_to_array(i)
  i_arr = np.expand_dims(i_arr, axis=0)
  i_arr = i_arr / 255.0
  pred = CDC.predict(i_arr)
  if pred[0] > 0.5:
    return 'The Image is identified as Dog'
  else:
    return 'The Image is identified as Cat'

In [94]:
cat_dog_ID = gr.Interface(
    fn=c_d_id,
    inputs=gr.Image(type='pil'),
    outputs=gr.Text(),
    title='Cat Dog Classifier',
    live=True
)

cat_dog_ID.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1195e08adefe4f30c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
